# encoding
convert text features to numerical
1. get "RecentTrainingExperience" from "RecentTrainingExperienceFrom" and "RecentTrainingExperienceTo"
2. combine team PokemonWorldRegion and PokemonTrainerClass
3. convert text features to numerical

In [10]:
import pandas as pd
import numpy as np
import datetime as dt

In [5]:
def read_csv(path_file_name):
    df = pd.read_csv(path_file_name, index_col=False)
    return df

In [29]:
path_file_name= "pokemon_trainer_application_data.csv" 
df= read_csv(path_file_name)
#print(df.shape)
df.head() 

/Users/h/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ApplicationStatus,ApplicationType,ApplyDate,CurrentlyEmployed,CurrentlyTrainingPokemon,CurrentlyWorkInThisRegion,GraduateSchoolGPA,GraduateSchoolGraduate,GymBadge1Pokemon,GymBadge2Pokemon,...,ReferenceRelationship3,SpokenWithCurrentBoss,TechnicalSchoolGPA,TechnicalSchoolGraduate,TotalYearsOfExp,UndergradSchoolGPA,UndergradSchoolGraduate,WorkedInThisRegion,WorkingForJobAppliedFor,hired
0,Filed Not Hired,Applicant,2013-11-26,False,NaN,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,False,NaN,0.0
1,Filed Not Hired,Applicant,2013-11-27,True,NaN,False,NaN,NaN,NaN,NaN,...,Fellow Trainer,NaN,NaN,NaN,5.0,NaN,No,False,NaN,0.0
2,Filed Not Hired,Applicant,2013-11-22,NaN,NaN,False,NaN,NaN,NaN,NaN,...,Fellow Trainer,NaN,NaN,NaN,5.0,3.0,Yes,False,NaN,0.0
3,Filed Not Hired,Applicant,2013-11-23,True,NaN,False,NaN,NaN,Seedot,NaN,...,Boss,NaN,NaN,Yes,1.0,2.4,No,False,NaN,0.0
4,New,Applicant,2013-11-23,True,NaN,False,NaN,NaN,NaN,NaN,...,Professor,NaN,NaN,NaN,3.0,3.0,No,False,NaN,NaN


#### *get "RecentTrainingExperience" from "RecentTrainingExperienceFrom" and "RecentTrainingExperienceTo"

In [30]:
def recent_train(df, col):
    day_to= col+'To'
    day_from= col+'From'
    df[col]= ( pd.to_datetime(df[day_to], dayfirst=True, errors='coerce') 
              -pd.to_datetime(df[day_from], dayfirst=True, errors='coerce') ).dt.days 
    # some feature is wrong e.g. df.at[99974, 'RecentTrainingExperience1To']= "2099-01-01"
    # remove days over 3600(10 years)
    df[col][df[col]>3600]= np.nan
    
    #delecte "RecentTrainingExperienceFrom" and "RecentTrainingExperienceTo"
    del df[day_to]
    del df[day_from]
    return df
    
df= recent_train(df, 'RecentTrainingExperience1')    
df= recent_train(df, 'RecentTrainingExperience2') 
df= recent_train(df, 'RecentTrainingExperience3') 

/Users/h/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### *team: combine  PokemonWorldRegion and PokemonTrainerClass

In [31]:
df['team']= '(' + df['PokemonWorldRegion']+ ', ' + df['PokemonTrainerClass'] + ')'

#### *convert text features to numerical

In [36]:
#find bool and object(string) features
def find_text_features(df):
    cols_object= df.select_dtypes(include='object').columns.tolist()
    cols_bool= df.select_dtypes(include='bool').columns.tolist()
    cols= cols_object + cols_bool
    return cols

#convert text features to numerical
def encodeing(df, col):
    cats= df[col].unique().tolist()
    d = { v: i+1 for i, v in enumerate(cats) if v is not np.nan}
    d[np.nan]= np.nan
    df[col] = [d.get(item) for item in df[col]]
    return df

cols= find_text_features(df)
for col in cols:   df= encodeing(df, col)
df.head()

,ApplicationStatus,ApplicationType,ApplyDate,CurrentlyEmployed,CurrentlyTrainingPokemon,CurrentlyWorkInThisRegion,GraduateSchoolGPA,GraduateSchoolGraduate,GymBadge1Pokemon,GymBadge2Pokemon,...,TotalYearsOfExp,UndergradSchoolGPA,UndergradSchoolGraduate,WorkedInThisRegion,WorkingForJobAppliedFor,hired,RecentTrainingExperience1,RecentTrainingExperience2,RecentTrainingExperience3,team
0,1.0,1,1,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,1.0,NaN,0.0,3288.0,NaN,1277.0,1.0
1,1.0,1,2,2.0,NaN,1.0,NaN,NaN,NaN,NaN,...,5.0,NaN,2.0,1.0,NaN,0.0,122.0,122.0,245.0,1.0
2,1.0,1,3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,5.0,3.0,3.0,1.0,NaN,0.0,NaN,91.0,184.0,1.0
3,1.0,1,4,2.0,NaN,1.0,NaN,NaN,2.0,NaN,...,1.0,2.4,2.0,1.0,NaN,0.0,1308.0,214.0,762.0,1.0
4,2.0,1,4,2.0,NaN,1.0,NaN,NaN,NaN,NaN,...,3.0,3.0,2.0,1.0,NaN,NaN,214.0,243.0,213.0,1.0


In [41]:
def load_file(df, path_load_file):
    print("load path: ", path_load_file)
    df.to_csv(path_load_file, index=False)


path_load_file= "pokemon_trainer_application_data_numerical.csv"
load_file(df, path_load_file)

load path:  pokemon_trainer_application_data_numerical.csv
